problem link: https://leetcode.com/problems/trips-and-users/description/?lang=pythondata

In [112]:
import pandas as pd

### Generating test cases

In [139]:
# trips=pd.read_csv('./test_data/p262_trips.csv',delimiter='|')
# trips=pd.read_csv('./test_data/p262_trips_v2.csv',delimiter='|')
trips=pd.read_csv('./test_data/p262_trips_v3.csv',delimiter='|')
trips

,id,client_id,driver_id,city_id,status,request_at
0,1111,1,10,1,completed,2013-10-01


In [140]:
trips.dtypes

id             int64
client_id      int64
driver_id      int64
city_id        int64
status        object
request_at    object
dtype: object

In [141]:
trips.columns

Index(['id', 'client_id', 'driver_id', 'city_id', 'status', 'request_at'], dtype='object')

In [142]:
# users=pd.read_csv('./test_data/p262_users.csv',delimiter='|')
# users=pd.read_csv('./test_data/p262_users_v2.csv',delimiter='|')
users=pd.read_csv('./test_data/p262_users_v3.csv',delimiter='|')
users

,users_id,banned,role
0,1,No,client
1,10,Yes,driver


In [143]:
users.dtypes

users_id     int64
banned      object
role        object
dtype: object

In [144]:
users.columns

Index(['users_id', 'banned', 'role'], dtype='object')

### Solving problem

In [145]:
#merging everyting
trips=trips.merge(users,how='left',left_on='client_id',right_on='users_id').merge(users,how='left',left_on='driver_id',right_on='users_id')
trips

,id,client_id,driver_id,city_id,status,request_at,users_id_x,banned_x,role_x,users_id_y,banned_y,role_y
0,1111,1,10,1,completed,2013-10-01,1,No,client,10,Yes,driver


In [146]:
#converting string to date
trips['request_at_v2']=pd.to_datetime(trips.request_at).dt.date  
trips

,id,client_id,driver_id,city_id,status,request_at,users_id_x,banned_x,role_x,users_id_y,banned_y,role_y,request_at_v2
0,1111,1,10,1,completed,2013-10-01,1,No,client,10,Yes,driver,2013-10-01


In [147]:
#getting filtered dates
import datetime
start_date = datetime.datetime(2013, 10, 1).date()
end_date = datetime.datetime(2013, 10, 3).date()

trips=trips[(trips.request_at_v2>=start_date) & (trips.request_at_v2<=end_date)]
trips

,id,client_id,driver_id,city_id,status,request_at,users_id_x,banned_x,role_x,users_id_y,banned_y,role_y,request_at_v2
0,1111,1,10,1,completed,2013-10-01,1,No,client,10,Yes,driver,2013-10-01


In [152]:
#excluding trips with banned users
trips=trips[(trips.banned_x=='No') & (trips.banned_y=='No')]
trips

,id,client_id,driver_id,city_id,status,request_at,users_id_x,banned_x,role_x,users_id_y,banned_y,role_y,request_at_v2


In [153]:
#getting totla requested trips
total_requested_trips=trips.request_at.value_counts().rename('total_request').reset_index()
total_requested_trips

,request_at,total_request


In [154]:
#getting canceled trips
canceled_trips=trips[(trips.status=='cancelled_by_client') | (trips.status=='cancelled_by_driver')].request_at.value_counts().rename('total_canceled').reset_index()
canceled_trips

,request_at,total_canceled


In [155]:
#getting calcelation rate
df_merged=total_requested_trips.merge(canceled_trips,how='left',on='request_at').rename(columns={'request_at':'Day'}).fillna(0)
df_merged['Cancellation Rate']=round(df_merged.total_canceled/df_merged.total_request,2)
df_merged

,Day,total_request,total_canceled,Cancellation Rate


In [156]:
df_merged[['Day','Cancellation Rate']]

,Day,Cancellation Rate


- Hints
  - Need to be cautious about date range
  - Need to exclude complete,cancel for banned users

### Preparing Submission function

In [161]:
import pandas as pd
import datetime

def trips_and_users(trips: pd.DataFrame, users: pd.DataFrame) -> pd.DataFrame:
    #merging everyting
    trips=trips.merge(users,how='left',left_on='client_id',right_on='users_id').merge(users,how='left',left_on='driver_id',right_on='users_id')
    
    #excluding trips with banned users
    trips=trips[(trips.banned_x=='No') & (trips.banned_y=='No')]

    #converting string to date
    trips['request_at_v2']=pd.to_datetime(trips.request_at).dt.date 

    #getting filtered dates
    start_date = datetime.datetime(2013, 10, 1).date()
    end_date = datetime.datetime(2013, 10, 3).date()
    trips=trips[(trips.request_at_v2>=start_date) & (trips.request_at_v2<=end_date)]
    
    #getting cancelation rate
    total_trips=trips.request_at.value_counts().rename('total_request').reset_index()
    canceled_trips=trips[(trips.status=='cancelled_by_client') | (trips.status=='cancelled_by_driver')].request_at.value_counts().rename('total_canceled').reset_index()
    df_merged=total_trips.merge(canceled_trips,how='left',on='request_at').rename(columns={'request_at':'Day'}).fillna(0)
    df_merged['Cancellation Rate']=round(df_merged.total_canceled/df_merged.total_request,2)

    return df_merged[['Day','Cancellation Rate']]
    


### Testing the submission function

In [169]:
# trips=pd.read_csv('./test_data/p262_trips.csv',delimiter='|')
# trips=pd.read_csv('./test_data/p262_trips_v2.csv',delimiter='|')
trips=pd.read_csv('./test_data/p262_trips_v3.csv',delimiter='|')
trips

,id,client_id,driver_id,city_id,status,request_at
0,1111,1,10,1,completed,2013-10-01


In [170]:
# users=pd.read_csv('./test_data/p262_users.csv',delimiter='|')
# users=pd.read_csv('./test_data/p262_users_v2.csv',delimiter='|')
users=pd.read_csv('./test_data/p262_users_v3.csv',delimiter='|')
users

,users_id,banned,role
0,1,No,client
1,10,Yes,driver


In [171]:
trips_and_users(trips=trips, users=users) 

,Day,Cancellation Rate
